# Intro
This notebook is part of a mini-series that aims to build a simple RAG system for learning purposes.

This notebook is a bit independent from others as it will just highlight some experiments done with
different indexing methods.

This notebook assumes:
- we have a dataset available of chunks of text data to index **with appropriate embeddings** `chunk_id (int) | text_chunk (str) | embedding (tensor(N))`.

We'd like now to implement obtain nearest-neighbour search index for document retrieval.

Plan:
- The first section implements a simple flat kNN solution and observes its speed
- The second section replaces the dummy approach with 2 more sophisticated approaches (Inverted file index, HNSW index)
- The third section shows how to evaluate approximate nearest neighbours search solutions independently of the questions datasets
- There could have been a fourth section which evaluates our end-to-end retrieval flow (question -> embedding -> document retrieval -> is the document relevant?), but this has been done in the [Step 1] with a simple Flat Index solution

In [91]:
import os
import timeit
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

# This library contains an implementation for all the index retrieval algorithms we'd like to try
import faiss

from datasets import load_dataset, load_from_disk

LOCAL_DATASET_FOLDER = "local_datasets"

# The number of searches that we'll time to measure speed improvements
N_SEARCH_TIMEIT = 500

# Load Corpus

In [3]:
CORPUS_DATASET_NAME = "wiki-data-chunked-recursive-CS300with_simple_embeddings"

corpus_dataset = load_from_disk(
    os.path.join(LOCAL_DATASET_FOLDER, CORPUS_DATASET_NAME)
)


In [4]:
corpus_dataset

Dataset({
    features: ['id', 'url', 'title', 'original_id', 'text_chunk', 'embedding'],
    num_rows: 271938
})

In [14]:
# Set some common parameters
dim = len(corpus_dataset[0]["embedding"]) # dimension
k = 5 # number of neighbours to retrieve for each query

# a random vector to query for testing
INDEX_TO_RETRIEVE_SINGLE_VECTOR = 53 #random
vector_in_index = np.array(
    [corpus_dataset[INDEX_TO_RETRIEVE_SINGLE_VECTOR]["embedding"]]
)
eps = 1e-2
vector_in_index_with_noise = vector_in_index + eps * np.random.random(dim)

# Section 1: Dummy Flat search

In [6]:
# We can use InnerProduct directly as our embeddings are already normalised
# Initialise the index, the FAISS package makes it super easy
flat_index = faiss.IndexFlatIP(dim)

In [7]:
# Luckily we can all load it at once here
all_vectors = np.array(corpus_dataset["embedding"])

In [8]:
# And add everything one shot
flat_index.add(all_vectors)

In [9]:
# Check that we can retrieve our random vector, search should be exact
result_ip, result_indices = flat_index.search(
    vector_in_index,
    k
)

# Index 53 should be first with a IP value of 1
print(result_ip, result_indices)

[[0.9999998  0.67321146 0.6577222  0.6150769  0.58417845]] [[    53  36103     52  36104 101932]]


In [21]:
# Check that we can retrieve our random vector even with small noise added
result_ip, result_indices = flat_index.search(
    vector_in_index_with_noise,
    k
)

# Index 53 should be first with a IP value of ~1
print(result_ip, result_indices)

[[1.0003568  0.6741253  0.65998363 0.62174714 0.5858568 ]] [[    53  36103     52  36104 101932]]


In [16]:
# Measure the speed of this solution
time_for_queries = timeit.timeit(
    lambda: flat_index.search(
        np.random.random((1,dim)),
        k
    ), number = N_SEARCH_TIMEIT
)

print(f"It takes the FlatIndex {time_for_queries:.2f}s to run {N_SEARCH_TIMEIT} random queries")
print(f"That's {(time_for_queries / N_SEARCH_TIMEIT):.3f}s per query")

It takes the FlatIndex 3.29s to run 500 random queries
That's 0.007s per query


# Section 2: Two faster - but not exhaustive - methods

## Inverted File Index

In [18]:
n_voronoi_cells = 100
n_voronoi_cells_visited_per_search = 3

# Initialise the index, the FAISS package makes it super easy
quantizer = faiss.IndexFlatIP(dim)
ivf_index = faiss.IndexIVFFlat(quantizer, dim, n_voronoi_cells)

# This simply finds the centroids to use
assert not ivf_index.is_trained
ivf_index.train(all_vectors[0:5000, :])
assert ivf_index.is_trained

In [19]:
# Actually add the vectors in the structure
ivf_index.add(all_vectors)

In [25]:
# Check that we can retrieve our random vector
# For an exact vector match, we expect the query to be assigned
# to the right centroid and hence always be returned
result_distances, result_indices = ivf_index.search(
    vector_in_index,
    k,
    params=faiss.SearchParametersIVF(
        nprobe=n_voronoi_cells_visited_per_search
    )
)

# now sure why but the index returns the distance instead of the IP value, so we expect 0 here
print(result_distances, result_indices)

[[0.         0.65357697 0.68455553 0.7698461  0.83164287]] [[    53  36103     52  36104 101932]]


In [26]:
# Check that we can retrieve our random vector
# with some perturbation.
# There's a chance the query lands in a different
# centroid area and get missed...
result_distances, result_indices = ivf_index.search(
    vector_in_index_with_noise,
    k,
    params=faiss.SearchParametersIVF(
        nprobe=n_voronoi_cells_visited_per_search
    )
)

# ... but we still retrieve it here.
print(result_distances, result_indices)

[[0.01200528 0.6644685  0.6927518  0.7692248  0.8410053 ]] [[    53  36103     52  36104 101932]]


In [29]:
time_for_queries = timeit.timeit(
    lambda: ivf_index.search(
        np.random.random((1,dim)),
        k,
        params=faiss.SearchParametersIVF(
            nprobe=n_voronoi_cells_visited_per_search
        )
    ), number = N_SEARCH_TIMEIT
)

print(f"It takes the IVFIndex {time_for_queries:.2f}s to run {N_SEARCH_TIMEIT} random queries")
print(f"That's {(time_for_queries / N_SEARCH_TIMEIT):.5f}s per query")

It takes the IVFIndex 0.17s to run 500 random queries
That's 0.00034s per query


## HNSW

In [34]:
# Initialise the index, the FAISS package makes it super easy
M = 16 # More info on M parameter: https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md
efSearch = 10
hnsw_index = faiss.IndexHNSWFlat(dim, M)

In [36]:
# Actually add the vectors in the structure, HNSW will take a little bit more time at construction time
hnsw_index.add(all_vectors)

In [37]:
# Check that we can retrieve our random vector
# Even for an exact vector match we're not certain
# to retrieve our vector back...
result_distances, result_indices = hnsw_index.search(
    vector_in_index,
    k,
    params=faiss.SearchParametersHNSW(
        efSearch=efSearch
    )
)

# ... but we're still getting it here
print(result_distances, result_indices)

[[0.         0.65357697 0.68455553 0.7698461  0.8316429 ]] [[    53  36103     52  36104 101932]]


In [39]:
# Check that we can retrieve our random distorted vector
result_distances, result_indices = hnsw_index.search(
    vector_in_index_with_noise,
    k,
    params=faiss.SearchParametersHNSW(
        efSearch=efSearch
    )
)

# ... same
print(result_distances, result_indices)

[[0.01200528 0.6644685  0.6927518  0.7692248  0.8410053 ]] [[    53  36103     52  36104 101932]]


In [41]:
time_for_queries = timeit.timeit(
    lambda: hnsw_index.search(
        np.random.random((1,dim)),
        k,
        params=faiss.SearchParametersHNSW(
            efSearch=efSearch
        )
    ), number = N_SEARCH_TIMEIT
)

print(f"It takes the HNSWIndex {time_for_queries:.2f}s to run {N_SEARCH_TIMEIT} random queries")
print(f"That's {(time_for_queries / N_SEARCH_TIMEIT):.5f}s per query")

It takes the HNSWIndex 0.12s to run 500 random queries
That's 0.00023s per query


# Section 3: How to evaluate the quality of our Approximate Nearest Neighbour search

Broadly to tune our parameters, we want to make sure that we can retrieve some vectors in the
index by querying the index with some of the exact same vectors, or with a few pertubations. (Index Recall)

For IVF, we have 2 main parameters to tune:
- the number of voronoi cells to create (creation time)
- the number of voronoi cells to visit when querying (query time)

For HNSW, we have 2 main params to tune as well (https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md):
- M, which broadly reflects the density of links inter-nodes in the graph (creation time)
- efSearch which correlates with the 'search depth' (query time)

We can perform a grid search to optimise for recall.

In [68]:
dataset_size = len(corpus_dataset)

In [93]:
def run_experiment_with_ivf_index(n_voronoi_cells: int, n_voronoi_visits: int, eps: float=0.0):
    # Run a quick experiment with n_voronoi_cells / n_voronoi_visits params to figure
    # out the index's recall with these parameters.
    # We can also use eps to distort the query vectors
    quantizer = faiss.IndexFlatIP(dim)
    ivf_index = faiss.IndexIVFFlat(quantizer, dim, n_voronoi_cells)
    ivf_index.train(all_vectors[0:10000, :])
    assert ivf_index.is_trained

    # Add vectors
    ivf_index.add(all_vectors)
        
    N_QUERIES = 500
    # pick a random vector from the dataset
    rand_inds = np.random.randint([dataset_size] * N_QUERIES)
    noise = eps * np.random.random((N_QUERIES, dim))
    vectors_in_index_distorted = np.array(
        corpus_dataset[rand_inds]["embedding"]
    ) + noise

    D,I = ivf_index.search(
        vectors_in_index_distorted,
        k,
        params=faiss.SearchParametersIVF(
            nprobe=n_voronoi_visits
        )
    )

    # Returns the number of first retrieved index that match the original
    num_matches = sum(I[:, 0]==rand_inds)

    recall = num_matches / N_QUERIES
    return recall 

# Test with on parameter pair
print("Test recall with n_voronoi_cells=100, n_voronoi_visits=1, we get:")
run_experiment_with_ivf_index(
    n_voronoi_cells=100,
    n_voronoi_visits=1,
    eps=0.1 # reasonable distortion
)
    

Test recall with n_voronoi_cells=100, n_voronoi_visits=1, we get:


0.844

In [97]:
best_recall_so_far = -1
best_n_voronoi_cells = None
best_n_voronoi_visits = None

# simple hand crafted grid search but you get the idea
for n_voronoi_cells, n_voronoi_visits in tqdm([
    (200, 1),
    (100, 1),
    (50, 1),
    (200, 3),
    (100, 3),
    (50, 3),
]):

    new_recall = run_experiment_with_ivf_index(
        n_voronoi_cells=n_voronoi_cells,
        n_voronoi_visits=n_voronoi_visits,
        eps=0.1 # reasonable distortion        
    )
    print(f"-- n_voronoi_cells={n_voronoi_cells}, n_voronoi_visits={n_voronoi_visits} yields recall={new_recall}")
    if new_recall > best_recall_so_far:
        best_recall_so_far = new_recall
        best_n_voronoi_cells = n_voronoi_cells
        best_n_voronoi_visits = best_n_voronoi_visits

print("Best params:")
print(f"{best_n_voronoi_cells=}")
print(f"{best_n_voronoi_visits=}")

print("yields:")
print(f"{best_recall_so_far=}")

# Obviously you'd want to take into account processing time as well  

 33%|██████████████████████████▎                                                    | 2/6 [00:00<00:00,  4.62it/s]

-- n_voronoi_cells=200, n_voronoi_visits=1 yields recall=0.792
-- n_voronoi_cells=100, n_voronoi_visits=1 yields recall=0.806


 50%|███████████████████████████████████████▌                                       | 3/6 [00:00<00:00,  4.48it/s]

-- n_voronoi_cells=50, n_voronoi_visits=1 yields recall=0.836


 67%|████████████████████████████████████████████████████▋                          | 4/6 [00:00<00:00,  4.28it/s]

-- n_voronoi_cells=200, n_voronoi_visits=3 yields recall=0.974


 83%|█████████████████████████████████████████████████████████████████▊             | 5/6 [00:01<00:00,  4.14it/s]

-- n_voronoi_cells=100, n_voronoi_visits=3 yields recall=0.97


100%|███████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.97it/s]

-- n_voronoi_cells=50, n_voronoi_visits=3 yields recall=0.98
Best params:
best_n_voronoi_cells=50
best_n_voronoi_visits=None
yields:
best_recall_so_far=0.98


In [89]:
def run_experiment_with_hnsw_index(M: int, efSearch: int, eps: float=0.0):
    # Run a quick experiment with M / efSearch params to figure
    # out the index's recall with these parameters.
    # We can also use eps to distort the query vectors
    hnsw_index = faiss.IndexHNSWFlat(dim, M)
    hnsw_index.add(all_vectors)
    
    N_QUERIES = 500
    # pick a random vector from the dataset
    rand_inds = np.random.randint([dataset_size] * N_QUERIES)
    noise = eps * np.random.random((N_QUERIES, dim))
    vectors_in_index_distorted = np.array(
        corpus_dataset[rand_inds]["embedding"]
    ) + noise

    D,I = hnsw_index.search(
        vectors_in_index_distorted,
        k,
        params=faiss.SearchParametersHNSW(
            efSearch=efSearch
        )
    )
    # Returns the number of first retrieved index that match the original
    num_matches = sum(I[:, 0]==rand_inds)

    recall = num_matches / N_QUERIES
    return recall 

# Test with on parameter pair
print("Test recall with M=5, efSearch=5, we get:")
run_experiment_with_hnsw_index(
    M=5,
    efSearch=5,
    eps=0.1 # reasonable distortion
)
# 50% is not great! Hence there's value in optimising our parameters.

Test recall with M=5, efSearch=5, we get:


0.52

In [99]:
best_recall_so_far = -1
best_n_voronoi_cells = None
best_n_voronoi_visits = None

# simple hand crafted grid search but you get the idea
for M, efSearch in tqdm([
    (10, 10),
    (20, 10),
    (40, 10),
    (10, 20),
    (20, 20),
    (40, 20),
]):

    new_recall = run_experiment_with_hnsw_index(
        M=M,
        efSearch=efSearch,
        eps=0.1 # reasonable distortion        
    )
    print(f"-- M={M}, efSearch={efSearch} yields recall={new_recall}")
    if new_recall > best_recall_so_far:
        best_recall_so_far = new_recall
        best_n_voronoi_cells = n_voronoi_cells
        best_n_voronoi_visits = best_n_voronoi_visits

print("Best params:")
print(f"{best_n_voronoi_cells=}")
print(f"{best_n_voronoi_visits=}")

print("yields:")
print(f"{best_recall_so_far=}")

# Obviously you'd want to take into account processing time as well, shown in TQDM output

 17%|█████████████▏                                                                 | 1/6 [00:04<00:20,  4.10s/it]

-- M=10, efSearch=10 yields recall=0.876


 33%|██████████████████████████▎                                                    | 2/6 [00:08<00:17,  4.47s/it]

-- M=20, efSearch=10 yields recall=0.946


 50%|███████████████████████████████████████▌                                       | 3/6 [00:17<00:18,  6.21s/it]

...

-- M=10, efSearch=20 yields recall=0.924


 83%|█████████████████████████████████████████████████████████████████▊             | 5/6 [00:26<00:05,  5.16s/it]

-- M=20, efSearch=20 yields recall=0.952


100%|███████████████████████████████████████████████████████████████████████████████| 6/6 [00:34<00:00,  5.72s/it]

-- M=40, efSearch=20 yields recall=0.986
Best params:
best_n_voronoi_cells=50
best_n_voronoi_visits=None
yields:
best_recall_so_far=0.986
